## Image Homogenizer

In [1]:
# Importing useful dependencies
import io
import os
import boto3
from PIL import Image

In [2]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [3]:
# This function scans a bucket folder, converts every non-PNG image it finds into PNG, uploads it back with the same name but .png extension, and deletes the original file.
def convert_images_to_png(bucket, prefix=""):
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):

            key = obj["Key"]

            if obj['Size'] == 0 and key.endswith("/"): # skip the folder itself
                continue

            if os.path.splitext(key)[1].lower() == ".png": # skip png images
                continue

            # New key with .png extension
            new_key = os.path.splitext(key)[0] + ".png"

            # Download the image
            resp = s3.get_object(Bucket=bucket, Key=key)
            body = resp["Body"].read()
            
            # Convert to PNG
            img = Image.open(io.BytesIO(body))
            buf = io.BytesIO()
            img.save(buf, format="PNG")
            buf.seek(0)

            # Upload the image back as PNG (replace original with .png)
            s3.upload_fileobj(buf, Bucket=bucket, Key=new_key, ExtraArgs={"ContentType": "image/png"})

            # Delete the old image
            s3.delete_object(Bucket=bucket, Key=key)

            print(f"Replaced: {key} -> {new_key}")

In [4]:
# Change the format of images to PNG
convert_images_to_png(bucket = "formatted-zone", prefix = "images/")

Replaced: images/image_1759415197916.jpg -> images/image_1759415197916.png
Replaced: images/image_1759415198090.jpg -> images/image_1759415198090.png
Replaced: images/image_1759415198272.jpg -> images/image_1759415198272.png
Replaced: images/image_1759415198417.jpg -> images/image_1759415198417.png
Replaced: images/image_1759415198540.jpg -> images/image_1759415198540.png
Replaced: images/image_1759415198675.jpg -> images/image_1759415198675.png
Replaced: images/image_1759415198801.jpg -> images/image_1759415198801.png
Replaced: images/image_1759415198932.jpg -> images/image_1759415198932.png
Replaced: images/image_1759415199061.jpg -> images/image_1759415199061.png
Replaced: images/image_1759415199214.jpg -> images/image_1759415199214.png
Replaced: images/image_1759415199336.jpg -> images/image_1759415199336.png
Replaced: images/image_1759415199478.jpg -> images/image_1759415199478.png
Replaced: images/image_1759415199604.jpg -> images/image_1759415199604.png
Replaced: images/image_17